In [286]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import geopandas as gpd
import numpy as np

# Transformers and Geopandas

<b>Note:</b> the sklearn transformer and the neural network transformer we'll mention later, and probably several other things named 'transformer' are all different things. 

## What is an SKLearn Transformer?

SKlearn transformers are classes that implement the `fit` and `transform` methods. They are used to preprocess data before feeding it to a model. In normal English, they are a step in data preparation pipelines that apply some type of clean-up step to the data. We use existing transformers all the time to impute, scale, or encode data.

We can also write our own custom transformers by extending the `BaseEstimator` and `TransformerMixin` classes from the `sklearn.base` module. This is useful when we need to apply some custom transformation that is not available in the existing transformers. As long as we provide the needed functionality, our custom transformer can be used in the same way as the built-in transformers.

## New - Pandas Got Better

Recently, I think in 2023, the usability of the pipeline transformers was improved a bit by allowing them to work with pandas dataframes, not only arrays. Dataframes are much easier to use as a human, so this will make steps using and tailoring transformers easier. In the examples later in Machine Learning, we assume we need an array, so we make arrays before starting the pipeline. We now have more flexibility to keep the data in that dataframe format longer, so we can handle it through the pipeline process with more ease. 

## What's a Pipeline?

A pipeline in sklearn is a sequence of steps that are applied to the data. In data science, we often need to load a large amount of data, and apply processing steps to that data in bulk to do things like impute, scale, or encode it. The pipeline is a way to automate this process so we can treat it as a group of steps, not a whole bunch of individual actions we need to manage bit by bit. 

In most cases, we will load our data, send it through the pipleline, and the output of that will go to the predictive modelling algorithm. All the preprocessing steps are done in the pipeline, so we can just focus on the model.

### Simple Pipeline Example

Here's a simple example of a pipeline - any data fed through this pipeline will have two steps applied to it - first the imputation (filling in blanks), then the scaling (making sure all the data is on the same scale). Each of these steps is a transformer - an object that we can make by extending some classes. These steps can be literally anything that we can imagine, as long as we meet the expectations of what a transformer needs (fit and transform methods) and we accept and return the data in the correct format. 

In [ ]:
from sklearn import pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipe = pipeline.Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])


We can also use the make_pipeline function to make a pipeline. It really doesn't matter, I personally never use make_pipeline, but it's there if you want it. The other way has the step name part built in, so that's a bit better.

In [ ]:
from sklearn.pipeline import make_pipeline

pipe_for_fake_chumps = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

#### Indentation

This is something of a personal preference, but I find it far easier to read for these types of things where the items are lined up as they are above. VS Code will usually do this pretty well by default, but it really is much more readable than listing them all in a line. 

In [288]:
df = pd.read_csv("../data/titanic_train.csv")
cat_cols = df.select_dtypes(include=['object']).columns
df.drop(cat_cols, axis=1, inplace=True)
df

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000
887,888,1,1,19.0,0,0,30.0000
888,889,0,3,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [289]:
pd_after = pipe.fit_transform(df)
pd.DataFrame(pd_after, columns=df.columns)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,-1.730108,-0.789272,0.827377,-0.592481,0.432793,-0.473674,-0.502445
1,-1.726220,1.266990,-1.566107,0.638789,0.432793,-0.473674,0.786845
2,-1.722332,1.266990,0.827377,-0.284663,-0.474545,-0.473674,-0.488854
3,-1.718444,1.266990,-1.566107,0.407926,0.432793,-0.473674,0.420730
4,-1.714556,-0.789272,0.827377,0.407926,-0.474545,-0.473674,-0.486337
...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,-0.207709,-0.474545,-0.473674,-0.386671
887,1.718444,1.266990,-1.566107,-0.823344,-0.474545,-0.473674,-0.044381
888,1.722332,-0.789272,0.827377,0.000000,0.432793,2.008933,-0.176263
889,1.726220,1.266990,-1.566107,-0.284663,-0.474545,-0.473674,-0.044381


#### Results

What happened here? The original data went through the two steps, each of which applied some transformation that changed the data:
<ul>
<li> The imputer found each empty value, and replaced it with the mean of the column. </li>
<li> The scaler did a two step scaling process: </li>
    <ul>
    <li> The 'fit' part allowed the scaler to learn the mean and standard deviation of the data. </li>
    <li> The 'transform' part applied the scaling to the data - setting the values to be "standardized" - mean of 0 and standard deviation of 1. </li>
    <li> <b> The details of this should be covered in stats, we're just worried about the step here. </b> </li>
    </ul>
</ul>

### Pipeline Details

The pipeline is a pretty simple concept at its core - it is a series of steps where data goes in one end, each step does whatever it does, and the result comes out the end. Each pipeline step has two parts defined:
<ul>
<li> Name - this is how we refer to the step. When making more complex pipelines, we may need this. We can also grab a step by name. </li>
<li> Action - this is the transformer that is applied to the data. Any configuration goes in the constructor call here. </li>
</ul>

Once created, it does those steps to that data in that order. This is really useful in a real life scenario where we likely have lots of data either in an incoming flow or regular batches to be processed. If we make a pipeline, then all the processing is handled there - we can capture the data in its original format and trust that our pipeline will do the right thing to it.

<b>Note:</b> there are ways to have a smarter pipeline, that can do things like process part of the data in one way and part in another (like discreet and continuous data) - that uses the same pipeline framework, but with a couple of slightly more complex tools. 

In [290]:
pipe.named_steps

{'imputer': SimpleImputer(), 'scaler': StandardScaler()}

## Exercise - Make a Simple Pipeline

Make a pipeline for the titanic data. Try to have it do the following things:
<ul>
<li> Fill in missing values with the median value of that column. (Impute) </li>
<li> Scale the data so all the columns are on the same scale. (Use Min-max scaler) </li>
</ul>

If you're feeling ok here, try to incorporate some other steps. Most work with little to no modifications to the code you need to write. If you search for "sklearn pipeline" the documentation page has a link to a User Guide, which is a pretty good article with examples, there are some different transformers in there that you can try out. You can also just google "sklearn pipeline transformers" and find one that you can try. For the most part, these are all related to preparing data for ML modeling, so the details aren't something we have worried about yet, but the act of using the transformers is identical no matter what they actually do. 

In [291]:
simp_pipe = pipeline.Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])
simp_res = simp_pipe.fit_transform(df)
pd.DataFrame(simp_res, columns=df.columns)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,0.000000,0.0,1.0,0.271174,0.125,0.000000,0.014151
1,0.001124,1.0,0.0,0.472229,0.125,0.000000,0.139136
2,0.002247,1.0,1.0,0.321438,0.000,0.000000,0.015469
3,0.003371,1.0,0.0,0.434531,0.125,0.000000,0.103644
4,0.004494,0.0,1.0,0.434531,0.000,0.000000,0.015713
...,...,...,...,...,...,...,...
886,0.995506,0.0,0.5,0.334004,0.000,0.000000,0.025374
887,0.996629,1.0,0.0,0.233476,0.000,0.000000,0.058556
888,0.997753,0.0,1.0,0.346569,0.125,0.333333,0.045771
889,0.998876,1.0,0.0,0.321438,0.000,0.000000,0.058556


### Why Do I Care?

For the moment, we don't need to care about the details of those steps, but we do want to use those mechanics to do our work. The steps that the pipeline does can be anything we want, so if we have some spatial cleanup, we can make a transformer to do that, and add it to the pipeline!

In machine learning later on, we will use variations of these pipelines to load most of the data we use. It is even more seamless there, we load the data, then the pipeline feeds the output directly into the modelling algorithm - that wall of numbers version above is hidden from us.

### Pipeline Creation Options

There are a few ways to make a pipeline, and in general pipeline components can be connected together and swapped out as needed. This leans heavily on the duck typing pattern of thought we've been getting used to - we can swap things around, as long as it does what is required of it. 

## What Can Transformers Do?

Pretty much anything you want, some common examples include:
<ul>
<li> Imputing missing values </li>
<li> Scaling numerical features </li>
<li> Encoding categorical features </li>
<li> Extracting features from text </li>
<li> Reducing dimensionality </li>
</ul>

Basically anything we can express in a statement of "change the data in this way" can be a transformer.

### And How Do They Do It?

A transfomer is a pretty simple concept. It has two main methods:
<ul>
<li> `fit` - This method is used to learn the parameters of the transformation. For example, if we are scaling numerical features, the `fit` method will calculate the mean and standard deviation of each feature. </li>
    <ul>
    <li> In many cases, the `fit` method does nothing, it just returns 'self', but it is still required to be there. </li>
    <li> If we were imputing and inserting 'median' for missing values, this step would calculate the median of each feature. </li>
    </ul>
<li> `transform` - This method is used to apply the transformation to the data. For example, if we are scaling numerical features, the `transform` method will subtract the mean and divide by the standard deviation. </li>
</ul>

Each of these methods is automatically called by the sklearn pipeline when we insert them as steps. If we have any configuration, such as the number of features to extract or the method of scaling, we can pass these as arguments to the transformer's `__init__` method.

### Making a Customized Transformer

As long as we follow these expectations, we can make a pipeline step doing almost anything we want. We need to accept values in an expected format - a 2D array or dataframe, and return the values in that same correct format - the details can vary though, we can move, add, delete or change values, rows, and columns as we need. Each manipulation in the transform steps of transformers is basically a prescribed set of steps we could do to any spreadsheet of data. 

<b>Note:</b> the X copy thing isn't required, it is there as a safety thing to prevent accidental side effects. It's a generally good (or more accurately, safe) idea, but not a requirement. There's at least one solution below without it. The biggest risk here would be if you were to do something that changed the data in place, and then tried to use the original data later - for example, if your transformer has a sort, you don't want to accidentally sort the original data and then try to use it later. In normal usage in a pipeline, this isn't normally something that comes up. It is possible, and can be very confusing if it does. 

### Transformer TL;DR

So, we can make a transformer to do anything we want, we just need to follow some constraints:
<ul>
<li> We need to extend the BaseEstimator and TransformerMixin classes. </li>
<li> We need to implement the fit and transform methods. </li>
<li> We need to accept and return the data in the correct format - a 2D table in either DF or array. </li>
</ul>

In [292]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self  # The fit method typically does nothing for transformers
        # This is mainly used when there is a 'configuration' 
        # step that needs to be done before the transformation
        # For example, when scaling data with standardization, 
        # we'd need the mean and std of the data - that's calculated here.
    def transform(self, X):
        # Your transformation logic goes here
        X_transformed = X.copy()  # Copy the input DataFrame to avoid modifying the original
        ############## Your code here ##############
        return X_transformed

### Using Custom Transformers

Adding a custom transformer to a pipeline is identical to using a premade one - that's one of the big benefits of the interchangeability of objects we have with python, duck typing, and inheritance - we only need to provide a tiny portion of the functionality that differs, not learn and reimplement the entire thing!

We must conform strictly to the input and output format, as well as the required methods, but beyond that we can do whatever we want. For this example below, we are adding two steps - the area calculator and the custom one. Each step gets a name to refer to it from here on out, and each init call can contain any arguments that we need or want to provide for configuration. 

## Exercise - Make a Custom Transformer

Make a custom transformer that adds a column to the data. The column should be the product of x and y.

If that works, add an optional parameter that allows the user to add a constant that defaults to 1, this constant should be multiplied by the product of x and y.

If all that works, add the ability to specify the column names for x and y, and name the output column. 

<b>Use your transformer by calling fit_transform, as well as in a pipeline.</b>

In [299]:
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000, factor=0.5, noise=0.1)
df_xy = pd.DataFrame(X, columns=["x", "y"])
df_xy.head()

,x,y
0,0.035930,0.498203
1,0.196935,1.113397
2,-0.087446,1.007661
3,1.043012,0.064344
4,1.097012,0.027744


In [303]:
class XYmultiplier(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1, x_col="x", y_col="y", output_col="product"):
        self.factor = factor
        self.x_col = x_col
        self.y_col = y_col
        self.output_col = output_col
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.output_col] = X_transformed[self.x_col] * X_transformed[self.y_col] * self.factor
        return X_transformed

In [304]:
xy_result = XYmultiplier(factor=2).fit_transform(df_xy)
xy_result.head()

,x,y,product
0,0.035930,0.498203,0.035801
1,0.196935,1.113397,0.438534
2,-0.087446,1.007661,-0.176232
3,1.043012,0.064344,0.134223
4,1.097012,0.027744,0.060870


In [334]:
xy_pipe = pipeline.Pipeline([
    ("xy_multiplier", XYmultiplier(factor=2))
])
xy_pipe_result = xy_pipe.fit_transform(df_xy)
xy_pipe_result.head()

,x,y,product
0,0.035930,0.498203,0.035801
1,0.196935,1.113397,0.438534
2,-0.087446,1.007661,-0.176232
3,1.043012,0.064344,0.134223
4,1.097012,0.027744,0.060870


## Geopandas and Pipelines

Geopandas is built "on top of" pandas, adding the ability to handle geospatial data to the already powerful pandas data manipulation library. We can use geopandas dataframes exactly as we would use a regular pandas dataframe, but with the added ability to handle geospatial data. Since geopandas dataframes are just pandas dataframes with some extra functionality, we can use them in sklearn pipelines as well.

There's little, if any, difference in when we make data a geodataframe. As a rough rule of thumb, if there's lots of manipulation to do with data, I'll load it as a pandas dataframe and make a geodataframe with that data; if there is not much processing, I'll read it with GeoPandas directly and set the geometry right up front. 

In [305]:
gpd_sample = gpd.read_file( "../data/03_city_edmonton_assessment_sample.csv", geometry="geometry")
gpd_sample.head()

,field_1,Suite,House Number,Street Name,Latitude,Longitude,Neighbourhood,Actual Year Built,Garage,Lot Size,Assessed Value,geometry
0,3407788,201.0,10111,160 STREET NW,53.54247,-113.59791,BRITANNIA YOUNGSTOWN,1980,False,99.029,55500,None
1,2636420,,0,,53.54485,-113.50787,DOWNTOWN,1922,False,1.69,16500,None
2,3635417,19,1407,GLASTONBURY BOULEVARD NW,53.49635,-113.6664,GLASTONBURY,2010,True,457.215,403500,None
3,3683433,,0,,53.43562,-113.58603,SOUTH TERWILLEGAR,2008,False,3.061,4000,None
4,1972487,,5223,111A STREET NW,53.48958,-113.51765,LENDRUM PLACE,1966,True,508.346,375500,None


In [306]:
pandas_sample = pd.read_csv("../data/03_city_edmonton_assessment_sample.csv")
pandas_sample = gpd.GeoDataFrame(pandas_sample, geometry=gpd.points_from_xy(pandas_sample["Latitude"], pandas_sample["Longitude"]))
pandas_sample.head()

,Unnamed: 0,Suite,House Number,Street Name,Latitude,Longitude,Neighbourhood,Actual Year Built,Garage,Lot Size,Assessed Value,geometry
0,3407788,201.0,10111,160 STREET NW,53.54247,-113.59791,BRITANNIA YOUNGSTOWN,1980,False,99.029,55500,POINT (53.542 -113.598)
1,2636420,NaN,0,NaN,53.54485,-113.50787,DOWNTOWN,1922,False,1.690,16500,POINT (53.545 -113.508)
2,3635417,19,1407,GLASTONBURY BOULEVARD NW,53.49635,-113.66640,GLASTONBURY,2010,True,457.215,403500,POINT (53.496 -113.666)
3,3683433,NaN,0,NaN,53.43562,-113.58603,SOUTH TERWILLEGAR,2008,False,3.061,4000,POINT (53.436 -113.586)
4,1972487,NaN,5223,111A STREET NW,53.48958,-113.51765,LENDRUM PLACE,1966,True,508.346,375500,POINT (53.490 -113.518)


### Realistic(-ish) Examples

Here are a couple of simple examples. The first adds an area column, the second generates points from lat/lon and places that into the geometry column. 

In [296]:
class areaGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X["area"] = self.area
        return X

In [297]:
class PointGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["geometry"] = gpd.points_from_xy(X_transformed[self.lon], X_transformed[self.lat])
        return X_transformed

## Example - Spatial Join Transformer

In this example, we can use a spatial join to connect two datasets based on their location. Specifically, we can use the point position of items in our data to connect with the area polygons in a spatial dataset, and get the neighborhood label from that spatial dataset. To make this work, we'll need a few parts:
<ul>
<li> A geopandas dataframe with the spatial data. This is like a setting or configuration step, so it will be in the constructor. </li>
<li> A transformer that can take the 'regular' data we're using, perform the spatial join, and add the result in a new column. </li>
</ul>

As long as our input and outputs match the sklearn transformer format, we can use this in a pipeline just like any other transformer.

In [307]:
class Spatial_Joiner( BaseEstimator, TransformerMixin):
    # Still must complete. 
    def __init__(self, other):
        self.other = other
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = gpd.sjoin(X, self.other, how="left", op="contains")
        return X

##### Prepare Data

I'll prepare the data here. The main thing is to ensure that geometry has properly captured the spatial data from the file. In this example, each column requires a little processing to get it into the right format.

In [308]:
booze = gpd.read_file('../data/Alcohol_Sales_Licences.csv', crs=6933)
booze['geometry'] = gpd.points_from_xy(booze['Longitude'], booze['Latitude'])
booze.set_geometry('geometry', inplace=True, crs=6933)

hoods = gpd.read_file('../data/Neighbourhood_Boundaries.csv')
hoods["geometry"] = gpd.GeoSeries.from_wkt(hoods["geom"])
hoods.set_geometry("geometry", inplace=True, crs=6933)
hoods.drop("geom", axis=1, inplace=True)

In [309]:
joinPipe = pipeline.Pipeline([
    ("joiner", Spatial_Joiner(other=booze))
])
tran_join_result = joinPipe.fit_transform(hoods)
tran_join_result.sample(10)

/Users/akeem/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/base.py:859: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  return self.fit(X, **fit_params).transform(X)


,Neighbourhood Number,Friendly Name,Description,geometry,index_right,Name,Business Description,Category,Zoning,Address,Neighbourhood ID,Latitude,Longitude,Point
74,6790,Wild Rose,Wild Rose is named for Alberta's official flow...,"MULTIPOLYGON (((-113.378 53.477, -113.377 53.4...",47.0,LIQUOR ON 17,Liquor store,Alcohol Sales (Consumption Off Premises),CB1,1758 - 34 AVENUE NW,6790,53.46801323521445,-113.3706512,POINT (-113.370651218797 53.468013235214)
67,3050,Brown Industrial,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.577 53.584, -113.577 53.5...",176.0,HAPPY HOUR LIQUOR,Liquor store,Alcohol Sales (Consumption Off Premises),"IB, IM","13, 14220 - YELLOWHEAD TRAIL NW",3050,53.58184921683823,-113.5666969,POINT (-113.56669691701 53.581849216838)
131,4486,Stewart Greens,,"MULTIPOLYGON (((-113.677 53.538, -113.677 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,4487,Secord,"Secord is a neighbourhood in west Edmonton, Al...","MULTIPOLYGON (((-113.714 53.527, -113.714 53.5...",213.0,WESTGATE LIQUOR STORE,liquor sales,Alcohol Sales (Consumption Off Premises),CHY,21621 - STONY PLAIN ROAD NW,4487,53.540614907193515,-113.6913588,POINT (-113.691358843858 53.540614907194)
115,4410,Poundmaker Industrial,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.653 53.556, -113.644 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,6530,Minchau,"Minchau was named in honour of August Minchau,...","MULTIPOLYGON (((-113.406 53.469, -113.406 53.4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,4710,Potter Greens,Potter Greens is a newer residential neighbour...,"MULTIPOLYGON (((-113.666 53.523, -113.666 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,3160,Dominion Industrial,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.568 53.581, -113.566 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,2390,Kernohan,Named for an early pioneer farmer who operated...,"MULTIPOLYGON (((-113.382 53.587, -113.382 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,4451,River Valley Oleskiw,This non-residential neighbourhood forms part ...,"MULTIPOLYGON (((-113.598 53.494, -113.598 53.4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Geospatial Transformations

With geospatial data, particularly for the things you're likely to be doing, these transformers and piplines can be used to make tools to process the data automatically into a format that provides what you need for analysis. We can create a pipeline that takes in raw data and outputs it in some format that we know we want - the steps to do those transformations are the transformers in the pipeline. Once built, we can process any new data with no additional effort by just running it through the pipeline - this means that we'd never do something like manually manipulate data in Excel or something like that, we'd always use the pipeline to do it for us.

For your applications, you'll get some data and the format you need it in might vary, or you likely might need more than one format. For example, if you were displaying some data in Tableau or Power BI, you can take some raw data, run it through a pipeline that calculates whatever values need to be displayed, then output that as a datasource for your visualization. The transformer step might do all kinds of stuff like calculate distance, do spatial joins to get region labels, calculate area, etc... This data can then be fed to the visualization tool, and it can make pretty pictures without having to manipulate data there. There can be multiple pipelines (or outputs) that each prepares one central source of data, automatically, for different purposes.

If this sounds similar to some ETL stuff you talked about in the database classes, that's because it is. 

In [310]:
class AddAreaCalculation( BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.column_name] = X_transformed["geometry"].area
        return X_transformed

## Exercise - Do That With Transformers

The processing steps above can be replaced with a pipeline containing custom steps in transformers. For the booze, we want to create a point from the lat/lon columns and set that answer into the geometry column. For the area, we want to add an area column.

<b>Hint:</b> we can chain transformers together almost any way we want, they can be connected like pipes in almost any configuration. The easy way to do this might not be the most intuitive... 

![Pipes](../images/pipe.jpg "Pipes")

In [321]:
class Point_Maker( BaseEstimator, TransformerMixin):
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["geometry"] = gpd.points_from_xy(X_transformed[self.lon], X_transformed[self.lat])
        X_transformed.set_geometry("geometry", inplace=True, crs=6933)
        return X_transformed
    
class Boundary_Processor( BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["geometry"] = gpd.GeoSeries.from_wkt(X_transformed["geom"])
        X_transformed.set_geometry("geometry", inplace=True, crs=6933)
        X_transformed.drop("geom", axis=1, inplace=True)
        return X_transformed


In [333]:
booze2 = gpd.read_file('../data/Alcohol_Sales_Licences.csv')
hoods2 = gpd.read_file('../data/Neighbourhood_Boundaries.csv')

booze_pipe = pipeline.Pipeline([
    ("point_maker", Point_Maker(lon="Longitude", lat="Latitude")),
])

clean_pipe = pipeline.Pipeline([
    ("boundary", Boundary_Processor()),
    ("join", Spatial_Joiner(other= booze_pipe.fit_transform(booze2)))
])
clean_data = clean_pipe.fit_transform(hoods2)
clean_data.sample(10)

/Users/akeem/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/base.py:859: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  return self.fit(X, **fit_params).transform(X)


,Neighbourhood Number,Friendly Name,Description,geometry,index_right,Name,Business Description,Category,Zoning,Address,Neighbourhood ID,Latitude,Longitude,Point
307,4470,Rural West Big Lake,This neighbourhood is rural in nature and prim...,"MULTIPOLYGON (((-113.689 53.582, -113.689 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,4475,Kinokamau Plains Area,,"MULTIPOLYGON (((-113.640 53.585, -113.640 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,2650,River Valley Rundle,This non-residential neighbourhood forms part ...,"MULTIPOLYGON (((-113.401 53.558, -113.400 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,3240,Inglewood,The name Inglewood was used on a 1905 plan of ...,"MULTIPOLYGON (((-113.548 53.559, -113.550 53.5...",113.0,SHOP RITE LIQUOR,Operate a liquor store,Alcohol Sales (Consumption Off Premises),CB1,11447 - 124 STREET NW,3240,53.56507785728897,-113.5354212,POINT (-113.535421196619 53.565077857289)
137,4630,White Industrial,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.656 53.563, -113.657 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,6780,Weir Industrial,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.394 53.516, -113.394 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,6360,King Edward Park,King Edward Park was farmed as early as the 18...,"MULTIPOLYGON (((-113.468 53.513, -113.469 53.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,1080,Delton,"Delton has been part of Edmonton since 1910, a...","MULTIPOLYGON (((-113.476 53.577, -113.478 53.5...",153.0,FRIENDS 97 LIQUOR,Alcohol Sales,Alcohol Sales (Consumption Off Premises),CSC,12455 - 97 STREET NW,1080,53.58086171297611,-113.4915188,POINT (-113.491518830326 53.580861712976)
331,8886,Edmonton South NBHD3,,"MULTIPOLYGON (((-113.467 53.367, -113.467 53.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,5380,River Valley Terwillegar,This non-residential neighbourhood forms part ...,"MULTIPOLYGON (((-113.591 53.498, -113.590 53.4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transformative Thoughts

As we make these transformers, there are a few things that we might want to consider more than we would in other scenarios:
<ul>
<li> Speed - these transfomrations may be run on all data, so efficiency might matter. </li>
    <ul>
    <li> We should avoid things that are obviously slow, like nested loops. </li>
    <li> Vectorization is a good goal - like np operations on arrays. </li>
    </ul>
<li> Safety - we should be careful to avoid side effects. </li>
    <ul>
    <li> We should avoid changing the input data in place as it introduces risk. Unlikely, but possible. </li>
    <li> If we make a transformer, we have 0 idea where it'll be used, so safety first. </li>
    <li> It isn't relevant right now, but it isn't uncommon for us to process some data and send it to a model, then use that data for something else like visualizations or another model. We don't want accidental changes, that are invisible, to the data behind our backs. In most cases, this won't be a real issue, but it is a very hard problem to trouble shoot if it does happen. </li>
    </ul>
<li> Flexibility - we should make sure that the transformer can handle a variety of data. By defult pipelines can work with data in arrays and dataframs in a way that is more-or-less seamless. We want the same behaviour in most cases. This usually isn't that big of a stretch, most other functions that we might need will work with different data types already, so as long as we don't introduce some object-specific action, we should be ok. </li>
</ul>

## Exercise - Make Some Transformers

For this exercise, create a transformer that adds a new column representing the distance between the two locations in the dataframe. The transformer should take the two columns containing the latitude and longitude of the two locations as input and add a new column with the distance between them.

Create a pipeline with transformers that:
<ul>
<li> Adds a new column with the distance to a set point. </li>
<li> Adds a new column with the distance to the nearest point in another dataset. </li>
<li> Adds a new column 'geometry' that is the point geometry, from lat and lon. </li>
<li> Add a transformer that filters things that are within a set distance of a point. </li>
</ul>

<b>Editor's Note:</b> I wrote these first, and they ended up being a bit redundant. I just didn't delete these prompts, but there's some overlap with the stuff above. 

In [325]:
class Add_Distance_Transformer( BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        X_transformed = X.copy()  # Copy the input DataFrame to avoid modifying the original
        #X_transformed["distance"]
        return X_transformed

In [326]:
class DifferenceFromSetPoint( BaseEstimator, TransformerMixin):
    def __init__(self, lon, lat, crs=6933):
        self.lon = lon
        self.lat = lat
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["distance"] = gpd.distance(X_transformed["geometry"], gpd.points_from_xy(X_transformed[self.lon], X_transformed[self.lat]))
        return X_transformed

In [327]:
class ClosestInOtherDataset( BaseEstimator, TransformerMixin):
    def __init__(self, gdf, column_name):
        self.gdf = gdf
        self.column_name = column_name
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # find the closest point in the other dataset
        # using column name as the column to compare
        X_transformed = X.copy()
        X_transformed = gpd.sjoin(X_transformed, self.gdf, how="left", op="intersects")
        # get the distance between the two points
        X_transformed["distance"] = gpd.distance(X_transformed["geometry"], X_transformed["geometry_2"])
        # get the closest point
        X_transformed = X_transformed.sort_values("distance").groupby("index_right").first()
        return X_transformed
    

In [328]:
class AddPointFromLatLon( BaseEstimator, TransformerMixin):
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["geometry"] = gpd.points_from_xy(X_transformed[self.lon], X_transformed[self.lat])
        X_transformed = gpd.GeoDataFrame(X_transformed, geometry="geometry")
        return X_transformed

In [329]:
ex_play = pd.read_csv("../data/playgrounds.csv")
ex_play.drop(columns=["Location", "Geometry Point"], inplace=True)
ex_play.head()

,ID,Playground Number,Name,Address,Type,Surface Type,Accessibility,Owner,Maintainer,Redevelopment Date,Latitude,Longitude,Neighbourhood ID,park_type,User Category
0,12504,631,Queen Alex Spray Park,10722 - 73 AVENUE NW,Spray Deck,Pour In Place Rubber,Wheelchair Accessible,Parks,Parks,2010-09-01,53.509733,-113.507833,5330,spray,NaN
1,708289,3677,Cy Becker Spray Park,270 Cy Becker Boulevard NW,Spray Park,Concrete,Wheelchair Accessible,Parks,Parks,2018-09-14,53.634241,-113.407026,2611,spray,NaN
2,12359,728,King Edward Spray Park,7708 - 85 STREET NW T6C4K9,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2006-09-01,53.513639,-113.460902,6360,spray,NaN
3,12513,636,Royal Gardens Spray Park,4030 117 Street NW,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2001-09-13,53.478097,-113.535659,5430,spray,NaN
4,708360,3684,York Spray Park,5825 140 Avenue NW,Spray Park,Concrete,Wheelchair Accessible,Parks,Parks,2018-09-14,53.602062,-113.431296,2720,spray,NaN


In [330]:
ex_pipe_play = pipeline.Pipeline([
    ("point", AddPointFromLatLon("Longitude", "Latitude")),
    ])
post_play = ex_pipe_play.fit_transform(ex_play)
post_play


,ID,Playground Number,Name,Address,Type,Surface Type,Accessibility,Owner,Maintainer,Redevelopment Date,Latitude,Longitude,Neighbourhood ID,park_type,User Category,geometry
0,12504,631,Queen Alex Spray Park,10722 - 73 AVENUE NW,Spray Deck,Pour In Place Rubber,Wheelchair Accessible,Parks,Parks,2010-09-01,53.509733,-113.507833,5330,spray,NaN,POINT (-113.50783 53.50973)
1,708289,3677,Cy Becker Spray Park,270 Cy Becker Boulevard NW,Spray Park,Concrete,Wheelchair Accessible,Parks,Parks,2018-09-14,53.634241,-113.407026,2611,spray,NaN,POINT (-113.40703 53.63424)
2,12359,728,King Edward Spray Park,7708 - 85 STREET NW T6C4K9,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2006-09-01,53.513639,-113.460902,6360,spray,NaN,POINT (-113.46090 53.51364)
3,12513,636,Royal Gardens Spray Park,4030 117 Street NW,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2001-09-13,53.478097,-113.535659,5430,spray,NaN,POINT (-113.53566 53.47810)
4,708360,3684,York Spray Park,5825 140 Avenue NW,Spray Park,Concrete,Wheelchair Accessible,Parks,Parks,2018-09-14,53.602062,-113.431296,2720,spray,NaN,POINT (-113.43130 53.60206)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,382,3690,Maple 5,2770 Maple Way NW,Playground,Engineered Wood Fibre,Wheelchair Accessible,NaN,Parks,2015-12-31,53.480627,-113.351146,6441,play,Junior (18 months to 5 years),POINT (-113.35115 53.48063)
731,861,1673,Dermott Playground,8610 81 Street NW,Playground,Engineered Wood Fibre,Unknown,NaN,Parks,2019-10-03,53.522317,-113.453516,6320,play,Senior (6 to 12 years),POINT (-113.45352 53.52232)
732,891,1767,Westmount Back Alley Park Playground,10718 - 130 STREET NW T5M0Y9,Playground,Sand,Unknown,NaN,Parks,1996-12-31,53.552360,-113.547174,3440,play,Senior (6 to 12 years),POINT (-113.54717 53.55236)
733,358,344,Ogilvie Ridge Community,951 Ogilvie Boulevard NW,Playground,Engineered Wood Fibre,Wheelchair Accessible,NaN,Parks,2010-12-31,53.463227,-113.562009,5300,play,Junior/Senior (18 months to 12 years),POINT (-113.56201 53.46323)


In [331]:
class FilterOutsideDistanceKM( BaseEstimator, TransformerMixin):
    def __init__(self, lon, lat, distance_km):
        self.lon = lon
        self.lat = lat
        self.distance_km = distance_km
        self.compare_point = gpd.points_from_xy([self.lon], [self.lat])[0]
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed["dist"] = self.compare_point.distance(X_transformed["geometry"])
        X_transformed = X_transformed[X_transformed["dist"] < self.distance_km]
        return X_transformed

In [332]:

nait_lat = 53.57030
nait_long = -113.50087

pipeDist = pipeline.Pipeline([
    ("point", AddPointFromLatLon("Longitude", "Latitude")),
    ("filter", FilterOutsideDistanceKM(nait_long, nait_lat, .1))
])

#post_play = pipeDist.fit(ex_play)
#post_play = pipeDist.transform(ex_play)
post_play = pipeDist.fit_transform(ex_play)
post_play

,ID,Playground Number,Name,Address,Type,Surface Type,Accessibility,Owner,Maintainer,Redevelopment Date,Latitude,Longitude,Neighbourhood ID,park_type,User Category,geometry,dist
0,12504,631,Queen Alex Spray Park,10722 - 73 AVENUE NW,Spray Deck,Pour In Place Rubber,Wheelchair Accessible,Parks,Parks,2010-09-01,53.509733,-113.507833,5330,spray,NaN,POINT (-113.50783 53.50973),0.060966
2,12359,728,King Edward Spray Park,7708 - 85 STREET NW T6C4K9,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2006-09-01,53.513639,-113.460902,6360,spray,NaN,POINT (-113.46090 53.51364),0.069339
3,12513,636,Royal Gardens Spray Park,4030 117 Street NW,Spray Deck,Concrete,Wheelchair Accessible,Parks,Parks,2001-09-13,53.478097,-113.535659,5430,spray,NaN,POINT (-113.53566 53.47810),0.098548
4,708360,3684,York Spray Park,5825 140 Avenue NW,Spray Park,Concrete,Wheelchair Accessible,Parks,Parks,2018-09-14,53.602062,-113.431296,2720,spray,NaN,POINT (-113.43130 53.60206),0.076481
5,12576,826,Gallagher Spray Park,9325 - 97 AVENUE NW,Spray Deck,Pour In Place Rubber,Wheelchair Accessible,Parks,Parks,2011-12-31,53.535919,-113.471814,6070,spray,NaN,POINT (-113.47181 53.53592),0.045014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,220,400,McCauley School,9538 107 Avenue NW,Playground,Sand,Unknown,NaN,Parks,1995-12-31,53.553063,-113.484155,1140,play,Junior/Senior (18 months to 12 years),POINT (-113.48416 53.55306),0.024011
728,786,306,Lendrum Park,11335 57 Avenue NW,Playground,"Engineered Wood Fibre, Pour In Place Rubber, Sand",Wheelchair Accessible,NaN,Parks,2015-12-31,53.494230,-113.524176,5270,play,Junior/Senior (18 months to 12 years),POINT (-113.52418 53.49423),0.079560
729,392,394,Sifton School Park,4305 134 Avenue NW,Playground,Sand,Unknown,NaN,Parks,1996-12-31,53.594144,-113.409569,2710,play,Senior (6 to 12 years),POINT (-113.40957 53.59414),0.094363
731,861,1673,Dermott Playground,8610 81 Street NW,Playground,Engineered Wood Fibre,Unknown,NaN,Parks,2019-10-03,53.522317,-113.453516,6320,play,Senior (6 to 12 years),POINT (-113.45352 53.52232),0.067415
